In [1]:
# import liberies 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.nn.functional as F


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%matplotlib inline


c:\Users\box69\miniconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# read true label from csv file
true_train_df = pd.read_csv("./Data/Train/BP/BP_train.csv") 
true_test_df = pd.read_csv("./Data/Test/BP/BP_test.csv")


print(f"True_train_df.shape {true_train_df.shape}\nTrue_test_df.shape {true_test_df.shape}")

display(true_train_df.head())
display(true_test_df.head())


True_train_df.shape (23675, 3)
True_test_df.shape (5924, 3)


,sbp,dbp,ppg_file
0,150.547151,61.772101,PPG_0.csv
1,163.143056,69.585197,PPG_1.csv
2,146.190404,69.817763,PPG_2.csv
3,108.187815,62.953105,PPG_3.csv
4,123.129155,56.773179,PPG_4.csv


,sbp,dbp,ppg_file
0,140.392609,61.372617,PPG_23675.csv
1,155.646471,62.985388,PPG_23676.csv
2,104.366073,55.113680,PPG_23677.csv
3,135.039295,57.435782,PPG_23678.csv
4,172.028767,78.547104,PPG_23679.csv


In [9]:
# read extracted features from csv file
train_features_df = pd.read_csv("./train_features_df.csv")
test_features_df = pd.read_csv("./test_features_df.csv")

print(f"train_features_df.shape {train_features_df.shape}\ntest_features_df.shape {test_features_df.shape}")
display(train_features_df.describe().transpose())

train_features_df.shape (23675, 32)
test_features_df.shape (5924, 32)


,count,mean,std,min,25%,50%,75%,max
MeanNN,23675.0,92.770983,17.570055,41.178571,77.000000,96.000000,107.200000,144.000000
SDNN,23675.0,4.939729,5.610217,0.000000,0.966092,2.027588,7.732687,57.418839
RMSSD,23675.0,7.031552,8.221308,0.000000,1.303840,2.692582,11.085777,87.023427
SDSD,23675.0,7.364625,8.659586,0.000000,1.354006,2.811541,11.639212,90.576855
CVNN,23675.0,0.052293,0.058165,0.000000,0.010903,0.021535,0.081307,0.726821
CVSD,23675.0,0.074362,0.084805,0.000000,0.015103,0.028402,0.115965,1.101562
MedianNN,23675.0,93.335924,18.044786,41.000000,77.000000,97.000000,108.500000,143.000000
MadNN,23675.0,2.006880,2.639385,0.000000,0.741300,1.482600,2.223900,32.617200
MCVNN,23675.0,0.021186,0.027527,0.000000,0.007642,0.014976,0.023166,0.351142
IQRNN,23675.0,2.855301,4.217572,0.000000,1.000000,1.500000,3.000000,66.000000


In [10]:
# select top features
top_features = ['sampen', 'fuzzyen', 'svden', 'skewness', 'MadNN', 'incren', 'dispen', 'kurtosis', 'SpEn', 'cumulativeresidual']
len(top_features)

10